
# Beyond CFT: Attacks on the Network and Convergence 


So far we showed how gossip can withstand network imperfection. But what if the attacker deliberately splits and attacks the network?  


Beyond regular crashes, peer can behave in various ways violating the protocol: hide transactions, send bogus data, create Sybil entities etc.
The goal of a blockchain system is to withstand against a powerful adversary. 

To ensure that message will be seen by the peer, once the peer is back online it must fetch the data from the neighboring peers. But what if the neighboring nodes are malicious and will censor certain transactions?   

In the next notebooks we will cover techniques that help to detect/prevent malicious behaviour.



# Malicious gossip agent 

One of the goal of a blockchain system is to record transaction in a 'hard-to-tamper' way.

How can you achieve that in P2P settings?  
It is common in databases and blockchains to use cryptography to verify the integrities of the transactions.

Let's first create a malicious agent that will change the data of received transactions to split the network.  


In [1]:
# Initialize the experiment:
from p2psimpy.config import *
import networkx as nx
from p2psimpy.services.gossip import GossipService
from p2psimpy.simulation import BaseSimulation
from p2psimpy.services.message_producer import MessageProducer
from p2psimpy.services.connection_manager import BaseConnectionManager
import copy
import warnings
warnings.filterwarnings('ignore')

# We use limited messages of count 20
class LimitedMessageProducer(MessageProducer):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.times = kwargs.pop('times', 20)

    def run(self):
        yield self.env.timeout(self.init_timeout)
        for _ in range(self.times):
            self.produce_transaction()
            yield self.env.timeout(self.tx_interval)
            
# Load the previous experiment configurations
exper = BaseSimulation.load_experiment(expr_dir='crash_gossip')

Locations, topology, peer_services, serv_impl = exper

saved_topology = copy.deepcopy(topology)

# Need to update some of the service implementatioin classes and message producer.
serv_impl['BaseConnectionManager'] = BaseConnectionManager
serv_impl['MessageProducer'] = MessageProducer
serv_impl['GossipService'] = GossipService

## Define malicious agents 
Let's first add malicious nodes randomly: 

In [2]:
# Change peer to a malicious 
from itertools import groupby
from random import sample

frac_malicious_nodes = 0.3 # 30 % of malicious nodes


def assign_malicious_peers(topology, mal_frac):
    type_dict = nx.get_node_attributes(topology, 'type')
    inv_type_dict = {k: {j for j, _ in list(v)}
                                for k, v in groupby(type_dict.items(), lambda x: x[1])}
    mal_nodes = sample(list(inv_type_dict['peer']), 
                       int(frac_malicious_nodes * len(inv_type_dict['peer'])))
    for b in mal_nodes:
        type_dict[b] = 'malicious'
        
    nx.set_node_attributes(topology, type_dict, 'type')
    
assign_malicious_peers(topology, frac_malicious_nodes)

## Define malicious services 

We will inherit a malicious gossip service that will relay the gossip message to one half of the network and the other half a tempered message (with different data). 



In [3]:
from p2psimpy.messages import *
from p2psimpy.consts import TEMPERED

class MaliciousGossipService(GossipService):
    
    
    def handle_message(self, msg):
        # Store the original message localy 
        self.peer.store('msg_time', msg.id, self.peer.env.now)
        self.peer.store('msg_data', msg.id, msg.data)

        if msg.ttl > 0:
            # Rely message further, modify the message
            exclude_peers = {msg.sender} | self.exclude_peers
            
            # Send the original message to one half of the network, 
            selected = self.peer.gossip(GossipMessage(self.peer, msg.id, msg.data, msg.ttl-1,
                                                      pre_task=msg.pre_task, post_task=msg.post_task), 
                                        self.fanout//2, 
                                        except_peers=exclude_peers, 
                                        except_type=self.exclude_types)
            # Change the message and send it to the other half
            new_data = TEMPERED
            exclude_peers = exclude_peers | set(selected)
            self.peer.gossip(GossipMessage(self.peer, msg.id, new_data, msg.ttl-1, 
                                           pre_task=msg.pre_task, post_task=msg.post_task), 
                             self.fanout//2, 
                             except_peers=exclude_peers, 
                             except_type=self.exclude_types)

##  Add malicious type and services 

We deliberately keep malicious nodes uncrashable. 


In [4]:
gossip_config = peer_services['peer'].service_map['RangedPullGossipService']
serv_impl['RangedPullGossipService'] = GossipService



peer_services['malicious'] = PeerType(peer_services['peer'].config,
                                      {BaseConnectionManager:None,
                                       MaliciousGossipService: gossip_config}
                                     )

## Run simulation 

Let's see how malicious agents together with crashing nodes affect the message dissemination. 

In [5]:
serv_impl

{'BaseConnectionManager': p2psimpy.services.connection_manager.BaseConnectionManager,
 'LimitedMessageProducer': __main__.LimitedMessageProducer,
 'RandomDowntime': p2psimpy.services.disruption.RandomDowntime,
 'RangedPullGossipService': p2psimpy.services.gossip.GossipService,
 'MessageProducer': p2psimpy.services.message_producer.MessageProducer,
 'GossipService': p2psimpy.services.gossip.GossipService}

In [6]:
from p2psimpy.messages import GossipMessage

In [7]:
# Init Graph
sim = BaseSimulation(Locations, topology, peer_services, serv_impl)
sim.run(10_200)

# Analyze the storage data




## Message data

Let's see how this fraction of malicious nodes affected the network. 
We compare the received message with the original message, we report `True` if the message wasn't tampered `False` and otherwise. 



In [8]:
import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        client_id, msg_num = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_id]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()

    
df = get_gossip_table(sim, 'msg_data', message_data)
df

1      3      5      6     7      9      10     11     12     13  \
1    True   True   True  False  True   True   True   True   True  False   
2    True   True   True  False  True   True   True  False   True   True   
3    True   True   True   True  True   True   True   True  False  False   
4    True   True   True   True  True   True   True   True   True   True   
5    True   True   True  False  True  False   True   True   True   True   
6    True   True   True  False  True   True   True   True  False   True   
7    True   True   True   True  True   True   True   True   True   True   
8    True   True   True   True  True  False  False  False   True  False   
9     NaN  False   True   True  True   True  False  False   True   True   
10  False   True   True   True  True   True   True   True   True  False   
11   True   True   True  False  True  False   True   True   True  False   
12   True   True   True   True  True   True   True   True   True   True   
13   True   True   True  False  True   True  False   True  False  False   
14  False   True   True   True   NaN  False    NaN    NaN   True   True   
15   True    NaN   True   True  True   True   True  False    NaN    NaN   
16   True   True   True   True  True  False  False   True  False   True   
17   True  False   True   True  True   True    NaN   True   True   True   
18    NaN   True  False   True  True   True  False   True  False   True   
19    NaN  False   True   True  True  False   True   True   True   True   
20   True   True   True   True  True   True   True   True   True   True   

       14     17     18     19     20     22     23     24  
1    True   True   True   True  False   True   True   True  
2    True   True  False   True   True   True   True   True  
3    True   True   True   True   True   True   True   True  
4    True   True   True   True   True   True   True   True  
5    True   True   True   True   True   True   True   True  
6    True   True  False   True   True   True   True   True  
7    True   True   True   True   True   True   True   True  
8    True  False   True   True   True  False  False   True  
9     NaN   True  False   True   True   True   True   True  
10   True   True   True   True   True   True   True   True  
11   True   True   True   True  False   True   True   True  
12   True   True   True   True   True   True   True   True  
13   True   True  False   True   True   True   True   True  
14   True  False   True   True   True   True  False    NaN  
15    NaN   True   True  False    NaN   True    NaN    NaN  
16  False   True  False   True   True   True   True   True  
17   True   True   True   True   True   True   True   True  
18   True   True  False   True   True   True   True   True  
19   True   True   True    NaN   True   True  False  False  
20   True   True   True   True   True   True   True   True

In [9]:
df[df==False].count()

1     2
3     3
5     1
6     6
7     0
9     6
10    5
11    4
12    5
13    6
14    1
17    2
18    6
19    1
20    2
22    1
23    3
24    1
dtype: int64

Malicious nodes managed to trick some peers into accepting wrong data! As peers will write 'first-seen' value, adversary once having advantage over the network can perfectly split the network. 

How to deal with this?

# Signing messages

First of all, messages themselves must be verified on their **integrity** and **authenticity**. 
[Digital signatures](https://en.wikipedia.org/wiki/Digital_signature) are perfect match for this and hence all blockchain systems use them. 

We will modify the code to simulate the signed messages. We will not use an actual crytpographic protocol since we care about only two things for our simulation: 
- It takes time to verify and sign messages. 
- Peers should store and forward only valid messages. 



## Simulating digital signatures 

We will show an example by building a crypto validator for 456 bits [EdDSA (Ed448)](https://en.wikipedia.org/wiki/EdDSA) (one of the most popular digital signatures in the wild).
On a regular laptop it takes usually less than 1 millisecond to verify a signature. Let's take the near worse case.  


This is not real. 

We will integrate a verification task into the message itself. 
Peer before triggering other services will first run the `pre_task`.  

Since the message is first created by MessageProducer we will add a task in the configuration.



In [11]:
from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist

conf = peer_services['client'].service_map['LimitedMessageProducer']

def validate_task(msg, peer):
    gen_dist = Dist('norm', (1, 0.2)) # time it takes to verify the message

    yield peer.env.timeout(gen_dist.get())
    if msg.data == TEMPERED:
        # You can decide what to do in this case.
        return False
    return True


class MsgConfig(Config):
    pre_task = Func(validate_task)
    init_ttl = 3
    # init_ttl = conf.init_ttl if conf else 3
    
    
peer_services['client'].service_map['LimitedMessageProducer'] = MsgConfig

In [14]:
# Run the simulation with a modificed message producer 
sim2 = BaseSimulation(Locations, topology, peer_services, serv_impl)
sim2.run(10_200)


In [15]:
df = get_gossip_table(sim2, 'msg_data', message_data)
df

1     3     5     6     7     9     10    11    12    13    14    17  \
1   True  True  True  True  True  True  True  True  True  True   NaN  True   
2   True  True  True  True  True  True  True  True  True  True  True  True   
3   True  True  True  True  True  True  True  True  True  True  True  True   
4   True  True  True  True  True  True  True  True  True  True  True  True   
5    NaN  True  True  True  True  True  True  True  True  True  True  True   
6   True  True  True   NaN  True  True  True  True  True  True  True  True   
7   True   NaN  True   NaN  True  True   NaN   NaN   NaN  True  True  True   
8   True  True  True   NaN  True  True  True  True   NaN  True   NaN   NaN   
9    NaN  True  True  True  True   NaN   NaN  True   NaN  True  True  True   
10  True  True   NaN   NaN  True  True   NaN  True  True  True   NaN  True   
11  True  True  True  True  True  True   NaN  True  True  True  True  True   
12  True  True  True  True  True  True  True  True  True  True   NaN  True   
13  True  True  True  True  True  True  True  True  True  True  True  True   
14   NaN  True  True  True  True  True  True  True  True  True  True  True   
15  True  True   NaN  True  True  True  True  True  True  True  True  True   
16  True   NaN  True  True   NaN  True  True  True  True   NaN  True  True   
17  True  True  True  True  True  True   NaN  True  True   NaN   NaN  True   
18  True  True  True  True  True  True   NaN  True  True   NaN  True  True   
19   NaN  True  True  True  True   NaN  True   NaN  True  True   NaN  True   
20  True   NaN  True  True  True   NaN  True  True  True  True  True  True   

      18    19    20    22    23    24  
1   True  True  True  True  True  True  
2   True  True  True  True  True  True  
3   True  True  True  True  True  True  
4   True  True  True  True  True  True  
5   True  True  True  True  True  True  
6   True  True   NaN  True  True  True  
7   True  True  True  True  True  True  
8   True   NaN  True  True   NaN  True  
9   True  True  True  True   NaN  True  
10  True   NaN   NaN   NaN   NaN  True  
11  True  True  True  True  True  True  
12   NaN  True  True  True  True  True  
13   NaN   NaN  True  True  True  True  
14  True  True  True  True  True  True  
15  True  True  True   NaN  True  True  
16  True  True   NaN  True  True  True  
17  True  True  True  True  True   NaN  
18   NaN  True  True  True  True  True  
19  True  True  True  True  True  True  
20  True  True  True  True  True  True

In [16]:
sim2.peers[12].storage['msg_data'].txs

{'26_1': GossipMessage:IFKGXWAAQSBWXGVSMLMG,
 '26_2': GossipMessage:VUXLHNJLAAICVMJGGGOG,
 '26_3': GossipMessage:LLLWQZPSZGXXBKXRJKTU,
 '26_4': GossipMessage:SCVJETMGXFKQYLDNIFYG,
 '26_5': GossipMessage:LXTRIZARXGZZADYFNYFT,
 '26_6': GossipMessage:GUTMRIENSBHSOUVXWYXH,
 '26_10': GossipMessage:VKPCJWNEAONDPIGZAQRI,
 '26_11': GossipMessage:GUSMKGEQQJYQSPBMYZVB,
 '26_12': GossipMessage:WLWMLAJIPNOPGCNXWJPT,
 '26_13': GossipMessage:OVZONXFKOZPDEXKDNUBQ,
 '26_14': GossipMessage:TLGAYAHGAVGROESDKDKJ,
 '26_15': GossipMessage:AFDDWNEAQOXTQGDXNQUD,
 '26_16': GossipMessage:XGJGROBUVJPDBZYACRVF,
 '26_17': GossipMessage:SUEHYTWUQWZCLXRAUJTH,
 '26_18': GossipMessage:SZCEEXUCSIXOMKUQVVYC,
 '26_19': GossipMessage:TZUIHXJYRNGCXCZDJSHN,
 '26_20': GossipMessage:VZXALHTRHGVZNVPTEAII}

Now malicious nodes cannot change the message. They need to explore other attack strategies!
The malicious nodes can delay messages, hide them, freeriding in a gossip (only listening). 
Together with network attack this can create a dangerous combination. 

Let us consider a case where an honest node is surrounded by malicious nodes (all network connections are with malicious nodes) that will hide certain transactions. As a result, peer will not receive crucial transactions that might affect it's decision making process. This attack is also called **Eclipse attack**.   

In reality, almost nothing stops one malicious node from running multiple instances and poison the whole network. This attack is called **Sybil Attack**. 




### Exercises


- Explore the limits of the gossip protocol. What is the maximum number of malicious nodes a protocol can tolerate? 
- Try to eclipse attack some peer, make sure he doesn't get any message, or one specific message (censor)? 



In [17]:
# According to our observation, the Gossip protocol is limited when the number of malicious nodes are equal to the minimum node degree in the topology.

def minimum_node_degree(topology):
    degrees = [val for (node, val) in topology.degree()]
    return min(degrees)
 
print("This topology and the Gossip protocol can tolerate {} malicious nodes".format(minimum_node_degree(topology) - 1)) 

This topology and the Gossip protocol can tolerate 3 malicious nodes


In [18]:
# Here, we will try to surround an honest node with all malicious nodes. 
# The job of malicious node is not to send a particular message, let's say message with id=2.

def create_eclipse_topo(topology):
    type_dict = nx.get_node_attributes(topology, 'type')
    inv_type_dict = {k: {j for j, _ in list(v)}
                                for k, v in groupby(type_dict.items(), lambda x: x[1])}
    degrees = {node:val for (node, val) in topology.degree()}
    minimum_degree_node = min(degrees, key=degrees.get)
    mal_nodes = [n for n in topology.neighbors(minimum_degree_node)]
    print("The honest node {} is surrounded by malicious nodes {}".format(minimum_degree_node, mal_nodes))
    print("The client is connected to ", [n for n in topology.neighbors(26)])
    for b in mal_nodes:
        type_dict[b] = 'malicious'
    #print(type_dict)
    nx.set_node_attributes(topology, type_dict, 'type')
    return minimum_degree_node
    
minimum_degree_node = create_eclipse_topo(saved_topology)

The honest node 16 is surrounded by malicious nodes [4, 7, 15, 18]
The client is connected to  [5, 15, 22, 24, 25]


In [19]:
from p2psimpy.messages import *
from p2psimpy.consts import TEMPERED

class MaliciousEclipseGossipService(GossipService):
    def __init__(self, *args, **kwargs):
        self.honest_node = kwargs.pop('honest_node', 0)
        self.cencor_msg_id = kwargs.pop('cencor_msg_id', 0)
        super().__init__(*args, **kwargs)
    
    def handle_message(self, msg):
        # Store the original message localy 
        self.peer.store('msg_time', msg.id, self.peer.env.now)
        self.peer.store('msg_data', msg.id, msg.data)

        if msg.ttl > 0:
            # Rely message further, modify the message
            # Cencor particular msg
            _, msg_num = msg.id.split('_')
            if int(msg_num) == self.cencor_msg_id:
                honest_peers = self.peer.sim.peers[self.honest_node]
                exclude_peers = {msg.sender} | self.exclude_peers | {honest_peers}
            else:
                exclude_peers = {msg.sender} | self.exclude_peers
            
            # Send the original message to one half of the network, 
            selected = self.peer.gossip(GossipMessage(self.peer, msg.id, msg.data, msg.ttl-1,
                                                      pre_task=msg.pre_task, post_task=msg.post_task), 
                                        self.fanout//2, 
                                        except_peers=exclude_peers, 
                                        except_type=self.exclude_types)
            # Change the message and send it to the other half
            new_data = TEMPERED
            exclude_peers = exclude_peers | set(selected)
            self.peer.gossip(GossipMessage(self.peer, msg.id, new_data, msg.ttl-1, 
                                           pre_task=msg.pre_task, post_task=msg.post_task), 
                             self.fanout//2, 
                             except_peers=exclude_peers, 
                             except_type=self.exclude_types)

In [20]:
gossip_config = peer_services['peer'].service_map['RangedPullGossipService']
serv_impl['RangedPullGossipService'] = GossipService

cencor_msg_id = 2
class EclipseGossipConfig(Config):
    exclude_types = {'client', }
    honest_node = minimum_degree_node
    cencor_msg_id = cencor_msg_id
    
peer_services['malicious'] = PeerType(peer_services['peer'].config,
                                      {BaseConnectionManager:None,
                                       MaliciousEclipseGossipService: EclipseGossipConfig}
                                     )

In [24]:
# Run the simulation with an Eclipse attack
# We will run the simulation for 10 times, and each time the honest node should return 'NaN' for a cencored msg_id = 2
for i in range(10):
    sim3 = BaseSimulation(Locations, saved_topology, peer_services, serv_impl)
    sim3.run(10_200)
    df = get_gossip_table(sim3, 'msg_data', message_data)
    print("Message id {} on honest node {} has value {}".format(cencor_msg_id,minimum_degree_node,df[minimum_degree_node][cencor_msg_id]))

Message id 2 on honest node 16 has value nan
Message id 2 on honest node 16 has value nan
Message id 2 on honest node 16 has value nan
Message id 2 on honest node 16 has value nan
Message id 2 on honest node 16 has value nan
Message id 2 on honest node 16 has value nan
Message id 2 on honest node 16 has value nan
Message id 2 on honest node 16 has value nan
Message id 2 on honest node 16 has value nan
Message id 2 on honest node 16 has value nan


In [25]:
# This confirms our Eclipse attack.